# Project
## First, I import the libraries, and extract the Most Actives ticker symbols.
## Second, I use Regular Expressions to locate the OPEN price, PREV CLOSE price, and VOLUME, writing into csv file, which is required in the project version 1.0. 
## Third, I convert the csv file into txt file, which is required in the project version 2.0. By the way, I also write to SQLite database from the csv file. I did not use the code in slides in which write to SQLite database from txt file, since I met some errors and can not resolve for several hours.


In [1]:
# Firstly, I need to import  libraries
from urllib.request import urlopen
import requests  # import requests library
import re    #use regular expressions 
import csv

        
#extract the ticker symbols     
cnnweb="https://money.cnn.com/data/hotstocks/"
hotstocks_handle = requests.get(cnnweb)
hotstocks_text = hotstocks_handle.text

active = hotstocks_text.find('Most Actives')
gainer = hotstocks_text.find('Gainers')

#finds Most Actives tickers 
most_actives_tickers = re.findall('.+class="wsod_symbol">(.*)</a>',hotstocks_text[active:gainer])

csv_file = [['Most Actives','Open price','Previous Close price','Volume']] # create the csv file, as required in the version 1.0


for q in range(len(most_actives_tickers)):
    csv_file.append([most_actives_tickers[q]])       #append the csv file 
    
    
# extract the stock ticker symbols and store them in a file or a list

yahoo_web = 'https://finance.yahoo.com/quote/{myticker}?p={myticker}&.tsrc=fin-srch-v1'
m=1
for ticker in most_actives_tickers:        
    mystock_handle=requests.get(yahoo_web.format(myticker = ticker))
    mystock_text = mystock_handle.text      # get data from a  website  
        
    open_price = re.findall('OPEN-value" data-reactid.*?>([0-9.]*)</span',mystock_text)   # use Regular Expressions to locate open price

    prev_close_price = re.findall('PREV_CLOSE-value" data-reactid.*?>([0-9.]*)</span',mystock_text)   # locate  PREV CLOSE price

    volume = re.findall('VOLUME-value" data-reactid.*?>([0-9.,]*)</span',mystock_text)          # locate VOLUME 
        
    csv_file[m] = csv_file[m] + open_price + prev_close_price + volume   # store the required columns into the csv file
    m = m+1
    
    
with open('stocks.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(csv_file)
csvFile.close()   


# write the csv file to a txt file
x='stocks.csv'
filehandle = open(x,'r')
#read all the lines in the csv file into a list called lines
lines = filehandle.readlines()
y='stocks.txt'
filehandle2 = open(y,'w')
filehandle2.writelines(lines)
filehandle2.close()  # close the file


# Write to SQLite database from a csv file. I did not use the code in slides writing into database from txt file, since I have tried and met errors which can not be solved for several hours 
import pandas
import csv, sqlite3
conn= sqlite3.connect("StocksDatabase.sqlite")
df = pandas.read_csv('stocks.csv')
df.to_sql('StocksTable', conn, if_exists='append', index=False)



C:\Users\hexiaqing\Anaconda33\lib\site-packages\pandas\core\generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)
